In [6]:
from gns3fy import Gns3Connector, Project
from build_topo import TopoBuilder
from ip_crud import set_ip, get_ip
import time
import multiprocessing

In [7]:
server = Gns3Connector('http://localhost:3080')
project = Project(name='test2', connector=server)
project.get()

In [8]:
#build topology
topo = TopoBuilder(server=server, project=project)
topo.build(9, 3, 3)


In [9]:
#get project
project.get()

#get vpc nodes to assign ip adddress
vpcses = [node for node in project.nodes if node.node_type == 'vpcs']

processes = []

#assign ip addresses to vpcs
for pc in vpcses:
    addr = f'192.168.1.{pc.name[-1]}'
    gtway = '255.0.0.0'
    process = multiprocessing.Process(target=set_ip, args=[pc, addr, gtway])
    processes.append(process)
    process.start()

#wait till all the processes run completely before going on
for process in processes:
    process.join()
    

In [11]:
#get project
project.get()

vpcs_ips = [get_ip(node) for node in project.nodes if node.node_type == 'vpcs']

for vpcs_ip in vpcs_ips:
    print(vpcs_ip)


192.168.1.1/8
192.168.1.2/8
192.168.1.3/8
192.168.1.4/8
192.168.1.5/8
192.168.1.6/8
192.168.1.7/8
192.168.1.8/8
192.168.1.9/8
